In [2]:
import import_ipynb
from downconverter import downconverter
#from lowpassFIR import lowpassFIR
#from lowpassIIR import lowpassIIR
import numpy as np
from scipy import signal

'''
INPUT
    mux_signal: multiplexed signal containing nb_audio audio signals in it
    Fs: sampling rate of the multiplexed signal
    audio_col: number of audio signals contained in the mux
    F_bw: multiplexing bandwidth (half the gap between carrier frequencies)
    filter_type: flag for filter type, can be either 0 for FIR or 1 for IIR
    filter_order: filter order
OUTPUT
    audio_output: a numpy array containing the demultiplexed audio signal, where each
    audio[:,i] contains one demultiplexed audio_signal for i going from 1
    to len(audio). Please have the index increase with carrier frequency
    (i.e. lowest carrier frequency corresponding to index 1 and highest
    carrier frequency corresponding to index len(audio)).
'''

def demux(mux_signal, Fs, audio_col, F_bw, filter_type, filter_order):
    # Write your implementation here
    fc_list = np.array([(2*0+1)*F_bw,(2*1+1)*F_bw,(2*2+1)*F_bw])
    

    signal_conv = []
    for i in range (audio_col):
      signal_1= mux_signal*(downconverter(mux_signal,len(mux_signal),Fs,fc_list[i]))
      signal_conv.append(signal_1)
    

    if filter_type ==1:
        filter_function = signal.butter(filter_order,2*F_bw/Fs)

    filtered_signals = []
    
    for k in range (audio_col):
            filtered_signal = signal.filtfilt(filter_function[0],filter_function[1],signal_conv[k])
            filtered_signals.append(filtered_signal)
    
    Normalizing = np.sqrt((np.sum(np.abs(mux_signal)**2))/len(mux_signal))
    filtered_signals = [signal/Normalizing for signal in filtered_signals]

    audio_output = np.zeros((len(filtered_signals[0]),audio_col))


    for g in range (audio_col):
        audio_output[:,g] = filtered_signals[g]
    separation = np.argsort(fc_list)
    audio_output = audio_output[:,separation]


    return audio_output.astype('float64')